Create a book recommendation algorithm using K-Nearest Neighbors.

Use the Book-Crossings dataset http://www2.informatik.uni-freiburg.de/~cziegler/BX/. This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users.

In [1]:
# import libraries 
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [4]:
# get data files
!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            
